# PureHDF

__[PureHDF](https://github.com/Apollo3zehn/PureHDF)__ is a .NET library implemented in C# for reading and writing HDF5 files. It is a pure .NET implementation of the HDF5 file format, and does not rely on any native libraries.  In other words, it is an independent (of the HDF5 library) implementation of an API and the HDF5 file format specification. It is based on the __[HDF5 File Format Specification Version 3.0 ](https://docs.hdfgroup.org/hdf5/develop/_f_m_t3.html)__.

<a href="https://en.wikipedia.org/wiki/.NET"><img src="img/Microsoft_.NET_logo.png" title=".NET logo" /></a>

C# code can be run in Jupyter notebooks using the __[dotnet-interactive](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode), aka `ms-dotnettools.dotnet-interactive-vscode`__ VS Code extension. Before installing the extension, you must install the .NET SDK 8.0 on your system.  The following instructions are for Debian 11, which      is the OS used for this container.

## Install the .NET SDK 8.0 on Debian 11

Run the folling commands in a terminal window: 

```bash
wget https://packages.microsoft.com/config/debian/11/packages-microsoft-prod.deb -O packages-microsoft-prod.deb
sudo dpkg -i packages-microsoft-prod.deb
rm packages-microsoft-prod.deb
sudo apt-get update && sudo apt-get install -y dotnet-sdk-8.0
```

## Install the VS Code extension

__[Polyglot Notebooks](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode)__, aka `ms-dotnettools.dotnet-interactive-vscode`

Finally install the __[PureHDF NuGet package](https://www.nuget.org/packages/PureHDF/)__  .

## Install the PureHDF NuGet package

Just execute the following command and the PureHDF NuGet package will be installed.

In [ ]:
#r "nuget: PureHDF, 1.0.0-beta.3"

We are now ready to use PureHDF in a Jupyter notebook.

## Let's go!

In [ ]:
using PureHDF;

For the generation of sample paths, we need  random numbers drawn from a normal distribution.  We could use the __[MathNet.Numerics](https://numerics.mathdotnet.com/)__ library for this purpose. However, we will just roll our own for the sake of simplicity. 

The code to generate the sample paths is almost identical to all the examples we have seen so far. Once that's out of the way, the conciseness of the C# code for writing the HDF5 entities is striking.

In [ ]:
ulong path_count = 100, step_count = 1000;
double dt = 0.01, theta = 1.0, mu = 0.0, sigma = 0.1;

// How do C# programmers get through life without the normal distribution?

var normal = (Random random, double mean, double stddev) =>
{
    double x1 = 1 - random.NextDouble();
    double x2 = 1 - random.NextDouble();
    double y1 = Math.Sqrt(-2.0 * Math.Log(x1)) * Math.Cos(2.0 * Math.PI * x2);
    return y1 * stddev + mean;
};

var random = new Random();

var data = new double[path_count, step_count];

for (ulong i = 0; i < path_count; i++)
{
    data[i, 0] = 0.0d;
    for (ulong j = 1; j < step_count; j++)
    {
        double dW = normal(random, 0.0, Math.Sqrt(dt));
        data[i, j] = data[i, j - 1] + theta * (mu - data[i, j - 1]) * dt + sigma * dW;
    }
}

var dataset = new H5Dataset(data, fileDims: [path_count, step_count]);

dataset.Attributes = new()
{
    ["comment"] = "This dataset contains sample paths of an Ornstein-Uhlenbeck process.",
    ["Wikipedia"] = "https://en.wikipedia.org/wiki/Ornstein%E2%80%93Uhlenbeck_process",
    ["rows"] = "path",
    ["columns"] = "time",
    ["dt"] = dt,
    ["θ"] = theta,
    ["μ"] = mu,
    ["σ"] = sigma
};

var file = new H5File()
{
    Attributes = new()
    {
        ["source"] = "https://github.com/HDFGroup/hdf5-tutorial"
    },

    ["dataset"] = dataset
};

file.Write("ou_purehdf.h5");

## Discussion

PureHDF is an independent implementation of HDF5 in C#/.NET. This is no small achievement and offers a fresh perspective on what’s possible with modern languages and platforms. If you thought that .NET was a Windows-only Microsoft thing, then this is your chance to experience a modern, free, cross-platform, open-source developer platform for building many different types of applications, and HDF5 is a first-class citizen.